In [ ]:
import torch,torchaudio,matplotlib.pyplot as plt,numpy as np,libnum
from scipy.signal import hilbert ,savgol_filter
wav,cyl = torchaudio.load('flag.mp3')
fw = wav[0]
hit = savgol_filter(np.abs(hilbert(fw.numpy())),500,1)
splitlist=[0]
hi = (hit[1:] - hit[:-1] )
i =0
while i < len(fw):
    if  len(np.where(hi[i-100:i]>=0)[0]) >80 and fw[i]>0.1 and (hit[i] -  hit[i-100]) / hit[i] >0.1:
        splitlist.append(i)
        i=int(i+cyl*0.1)
    i+=1
flag=''
for i in range(1,len(splitlist)-1):
    fft=torch.abs(torch.fft.fft(fw[splitlist[i]: splitlist[i+1]]))[:(splitlist[i+1]-splitlist[i])//2]
    maxs = torch.where(fft==fft.max())[0][0]//2
    if fft[maxs*5] < fft[maxs*4]:
        flag+='0'
    else :
        flag+='1'
print(libnum.b2s(flag))


In [ ]:
plt.figure(figsize=(20,3))
plt.ylim(-1,1)
plt.plot(hit[cyl*3:cyl*5] / hit.max()) 
plt.plot(fw[cyl*3:cyl*5]  ) 
cyl

In [ ]:
for i in splitlist:
    wav[0][i] = 1.5 
torchaudio.save('ff.wav',wav,cyl)

In [ ]:
stop = len(splitlist)-1
for i in range(0,10):
    print(i)
    fft=torch.abs(torch.fft.fft(fw[splitlist[i]: splitlist[i+1]]))[:(splitlist[i+1]-splitlist[i])//2]
    maxs = torch.where(fft==fft.max())[0][0]//2
    plt.figure(figsize=(15,2))
    plt.subplot(121)
    plt.plot(fft[:maxs*15])
    plt.scatter(maxs, fft[maxs]+10,c='r',marker='v')
    for x in range(1,15):
        plt.vlines(maxs*x, 0, fft[maxs] ,alpha=0.2,linestyles='dashed' ,colors='r') 
    plt.subplot(122)
    plt.plot(fw[splitlist[i]: splitlist[i+1]])
    plt.show()

